In [2]:
# | hidden: true
# | echo: false
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# some personal style settings to make the plots look nice
# and save some space in the notebook
plt.style.use("../style.mplstyle")

Now that we've talked about inference in depth, it is time to talk about **prediction**. In this lecture, we will cover the basics of linear regression, which is a powerful tool for making predictions based on data.